<a href="https://colab.research.google.com/github/jonesmo/data_science_projects/blob/main/EDA_part_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from numpy import NaN
import datetime
import requests

from google.colab import drive
drive.mount('/content/gdrive')

gresponses_df = pd.read_csv('/content/gdrive/My Drive/Thinkful/Experimental_Design_and_Testing/Capstone 1/govt_responses.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
gresponses_df = gresponses_df.assign(date_extended = pd.to_datetime(gresponses_df['Date'], format='%Y%m%d'))
gresponses_df['date_formatted'] = gresponses_df['date_extended'].dt.date

gresponses_df = gresponses_df.drop(['CountryCode', 'S2_Workplace closing','S1_Notes', 'S2_IsGeneral', 'S2_Notes', 'S3_Cancel public events', 'S3_IsGeneral', 'S3_Notes', 
                                    'S4_Close public transport', 'S4_IsGeneral', 'S4_Notes', 'S5_Public information campaigns', 'S5_IsGeneral',
                                    'S5_Notes', 'S6_Restrictions on internal movement', 'S6_IsGeneral', 'S6_Notes', 'S7_International travel controls', 'S7_Notes',
                                    'S8_Fiscal measures', 'S8_Notes', 'S9_Monetary measures', 'S9_Notes', 'S10_Emergency investment in health care',
                                    'S10_Notes', 'S11_Investment in Vaccines', 'S11_Notes', 'S12_Testing framework', 'S12_Notes',
                                    'S13_Contact tracing', 'S13_Notes', 'Unnamed: 39', 'date_extended', 'StringencyIndex', 'StringencyIndexForDisplay', 'ConfirmedCases'], axis=1)
# gresponses_df['new_deaths'] = 0

# for row in gresponses_df.iterrows():
#   gresponses_df['new_deaths'] = gresponses_df['ConfirmedDeaths'].diff()

gresponses_df.head()

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted
0,Aruba,20200101,0.0,NaN,NaN,2020-01-01
1,Aruba,20200102,0.0,NaN,NaN,2020-01-02
2,Aruba,20200103,0.0,NaN,NaN,2020-01-03
3,Aruba,20200104,0.0,NaN,NaN,2020-01-04
4,Aruba,20200105,0.0,NaN,NaN,2020-01-05


In [ ]:
gresponses_df['school_stringency'] = 0

for index, row in gresponses_df.iterrows():
  if row['S1_School closing'] == 0.0:
    gresponses_df.at[index, 'school_stringency'] = 0
  if row['S1_School closing'] == 1.0:
    if row['S1_IsGeneral'] == 0.0:
      gresponses_df.at[index, 'school_stringency'] = 1
    if row['S1_IsGeneral'] == 1.0:
      gresponses_df.at[index, 'school_stringency'] = 2
    if row['S1_IsGeneral'] == 3:
      gresponses_df.at[index, 'school_stringency'] = 2
  if row['S1_School closing'] == 2.0:
    if row['S1_IsGeneral'] == 0.0:
      gresponses_df.at[index, 'school_stringency'] = 3
    if row['S1_IsGeneral'] == 1.0:
      gresponses_df.at[index, 'school_stringency'] = 4
    if row['S1_IsGeneral'] == 3:
      gresponses_df.at[index, 'school_stringency'] = 4
  if row['S1_School closing'] == np.nan:
    gresponses_df.at[index, 'school_stringency'] = np.nan

gresponses_df['S1_IsGeneral'].fillna(1.0, inplace=True)
gresponses_df = gresponses_df[gresponses_df['S1_School closing'].notna()]
#gresponses_df = gresponses_df[gresponses_df['ConfirmedDeaths'].notna()]
gresponses_df = gresponses_df[gresponses_df['school_stringency'].notna()]

gresponses_df[1035:1045]

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency
1068,Azerbaijan,20200415,2.0,1.0,13.0,2020-04-15,4
1069,Azerbaijan,20200416,2.0,1.0,13.0,2020-04-16,4
1070,Burundi,20200101,0.0,1.0,NaN,2020-01-01,0
1071,Burundi,20200102,0.0,1.0,NaN,2020-01-02,0
1072,Burundi,20200103,0.0,1.0,NaN,2020-01-03,0
1073,Burundi,20200104,0.0,1.0,NaN,2020-01-04,0
1074,Burundi,20200105,0.0,1.0,NaN,2020-01-05,0
1075,Burundi,20200106,0.0,1.0,NaN,2020-01-06,0
1076,Burundi,20200107,0.0,1.0,NaN,2020-01-07,0
1077,Burundi,20200108,0.0,1.0,NaN,2020-01-08,0


In [ ]:
April_6_bools = gresponses_df['Date'] == 20200406
April_6 = gresponses_df.iloc[April_6_bools.values]
April_6

#the last date for which we have a large number of countries with complete information

#now I have to import the populations--used UN World Populations at https://population.un.org/wpp/Download/Standard/CSV/
un_populations = pd.read_csv('/content/gdrive/My Drive/Thinkful/Experimental_Design_and_Testing/Capstone 1/UN_populations.csv')
un_populations = un_populations[un_populations['Time'] == 2020]
un_populations = un_populations[un_populations['Variant'] == 'Medium']
un_populations = un_populations.drop(['LocID', 'VarID', 'Variant', 'MidPeriod', 'PopMale', 'PopFemale', 'Time'], axis=1)
un_populations = un_populations.dropna()
un_populations['PopTotal'] = un_populations['PopTotal'] / 100
un_populations

April_6 = April_6.merge(un_populations, how='inner', left_on='CountryName', right_on='Location')
April_6 = April_6.drop(['Location'], axis=1)
April_6

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity
0,Aruba,20200406,2.0,1.0,0.0,2020-04-06,4,1.06766,593.144
1,Afghanistan,20200406,2.0,1.0,7.0,2020-04-06,4,389.28341,59.627
2,Angola,20200406,2.0,1.0,2.0,2020-04-06,4,328.66268,26.363
3,Andorra,20200406,2.0,1.0,18.0,2020-04-06,4,0.77265,164.394
4,United Arab Emirates,20200406,2.0,1.0,10.0,2020-04-06,4,98.90400,118.306
...,...,...,...,...,...,...,...,...,...
117,Uruguay,20200406,2.0,1.0,6.0,2020-04-06,4,34.73727,19.848
118,Uzbekistan,20200406,2.0,1.0,2.0,2020-04-06,4,334.69199,78.677
119,South Africa,20200406,2.0,1.0,11.0,2020-04-06,4,593.08690,48.891
120,Zambia,20200406,2.0,1.0,1.0,2020-04-06,4,183.83956,24.730


##Add deaths per 100,000 population as a column

In [ ]:
April_6['Deaths_per_100,000'] = 0

for row in April_6.iterrows():
  April_6['Deaths_per_100,000'] = April_6['ConfirmedDeaths'] / April_6['PopTotal']

April_6

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity,"Deaths_per_100,000"
0,Aruba,20200406,2.0,1.0,0.0,2020-04-06,4,1.06766,593.144,0.000000
1,Afghanistan,20200406,2.0,1.0,7.0,2020-04-06,4,389.28341,59.627,0.017982
2,Angola,20200406,2.0,1.0,2.0,2020-04-06,4,328.66268,26.363,0.006085
3,Andorra,20200406,2.0,1.0,18.0,2020-04-06,4,0.77265,164.394,23.296447
4,United Arab Emirates,20200406,2.0,1.0,10.0,2020-04-06,4,98.90400,118.306,0.101108
...,...,...,...,...,...,...,...,...,...,...
117,Uruguay,20200406,2.0,1.0,6.0,2020-04-06,4,34.73727,19.848,0.172725
118,Uzbekistan,20200406,2.0,1.0,2.0,2020-04-06,4,334.69199,78.677,0.005976
119,South Africa,20200406,2.0,1.0,11.0,2020-04-06,4,593.08690,48.891,0.018547
120,Zambia,20200406,2.0,1.0,1.0,2020-04-06,4,183.83956,24.730,0.005440


##Now break into groups

In [ ]:
#then break them into groups: 0, 1, 2, 3, 4
group_0_bools = April_6['school_stringency'] == 0
group_0 = April_6.iloc[group_0_bools.values]
group_0

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity,"Deaths_per_100,000"
9,Burundi,20200406,0.0,1.0,0.0,2020-04-06,0,118.90781,463.037,0.000000
28,Cuba,20200406,0.0,1.0,8.0,2020-04-06,0,113.26616,106.413,0.070630
43,Gambia,20200406,2.0,1.0,1.0,2020-04-06,0,24.16664,238.801,0.041379
62,Kenya,20200406,2.0,1.0,4.0,2020-04-06,0,537.71300,94.478,0.007439
74,Mozambique,20200406,2.0,1.0,0.0,2020-04-06,0,312.55435,39.746,0.000000
77,Malawi,20200406,2.0,1.0,0.0,2020-04-06,0,191.29955,202.906,0.000000
82,Nicaragua,20200406,0.0,1.0,1.0,2020-04-06,0,66.24554,55.049,0.015095
100,Singapore,20200406,0.0,1.0,6.0,2020-04-06,0,58.50343,8357.633,0.102558
107,Sweden,20200406,0.0,1.0,401.0,2020-04-06,0,100.99270,24.612,3.970584


In [ ]:
group_2_bools = April_6['school_stringency'] == 2
group_2 = April_6.iloc[group_2_bools.values]
group_2

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity,"Deaths_per_100,000"
33,Dominican Republic,20200406,1.0,1.0,82.0,2020-04-06,2,108.47904,224.501,0.755906
60,Japan,20200406,1.0,1.0,73.0,2020-04-06,2,1264.76458,346.934,0.057718


In [ ]:
group_3_bools = April_6['school_stringency'] == 3
group_3 = April_6.iloc[group_3_bools.values]
group_3

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity,"Deaths_per_100,000"
6,Australia,20200406,2.0,0.0,36.0,2020-04-06,3,254.99881,3.319,0.141177
18,Brazil,20200406,2.0,0.0,486.0,2020-04-06,3,2125.59409,25.431,0.228642
24,China,20200406,2.0,0.0,3335.0,2020-04-06,3,14393.23774,153.312,0.231706
72,Myanmar,20200406,2.0,0.0,1.0,2020-04-06,3,544.09794,83.286,0.001838
98,Rwanda,20200406,2.0,0.0,0.0,2020-04-06,3,129.52209,525.019,0.000000


In [ ]:
group_4_bools = April_6['school_stringency'] == 4
group_4 = April_6.iloc[group_4_bools.values]
group_4

,CountryName,Date,S1_School closing,S1_IsGeneral,ConfirmedDeaths,date_formatted,school_stringency,PopTotal,PopDensity,"Deaths_per_100,000"
0,Aruba,20200406,2.0,1.0,0.0,2020-04-06,4,1.06766,593.144,0.000000
1,Afghanistan,20200406,2.0,1.0,7.0,2020-04-06,4,389.28341,59.627,0.017982
2,Angola,20200406,2.0,1.0,2.0,2020-04-06,4,328.66268,26.363,0.006085
3,Andorra,20200406,2.0,1.0,18.0,2020-04-06,4,0.77265,164.394,23.296447
4,United Arab Emirates,20200406,2.0,1.0,10.0,2020-04-06,4,98.90400,118.306,0.101108
...,...,...,...,...,...,...,...,...,...,...
117,Uruguay,20200406,2.0,1.0,6.0,2020-04-06,4,34.73727,19.848,0.172725
118,Uzbekistan,20200406,2.0,1.0,2.0,2020-04-06,4,334.69199,78.677,0.005976
119,South Africa,20200406,2.0,1.0,11.0,2020-04-06,4,593.08690,48.891,0.018547
120,Zambia,20200406,2.0,1.0,1.0,2020-04-06,4,183.83956,24.730,0.005440


##Now do Kruskal-Wallis on death rate for groups with stringency 0, 2, 3, and 4

In [ ]:
print(stats.kruskal(group_0['Deaths_per_100,000'], group_2['Deaths_per_100,000'], group_3['Deaths_per_100,000'], group_4['Deaths_per_100,000']))

KruskalResult(statistic=4.904254796533922, pvalue=0.17894383892899424)
